<h1> ShineBox Virtual Assistant </h1>

In [30]:
from google.colab import drive
import sys
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [31]:
sys.path.append("/content/drive/MyDrive/ShineBox")

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import unicodedata
import re
import numpy as np
import os
import io
import time
import random
from typing import Union

<h2>Prepering The Data</h2>

In [33]:

def merge_and_shuffle(file1, file2, output_file):
    with open(file1, 'r') as f1, open(file2, 'r') as f2:
        lines = f1.readlines()[:14000] + f2.readlines()[:1000]
    random.shuffle(lines)
    print("\n".join(lines[:5]))
    with open(output_file, 'w') as f:
        f.writelines(lines)


merge_and_shuffle("/content/drive/MyDrive/ShineBox/output.otl","/content/drive/MyDrive/ShineBox/jokes.otl", "/content/drive/MyDrive/ShineBox/data.otl")


input . ( do us passport holders need a visa for thailand ) => questionAndAnswer [ none ] . ( no )

input . ( episode where lily and marshall get back together ) => questionAndAnswer [ none ] . ( Atlantic City )

input . ( is negative infinity to positive infinity the same as all real numbers ) => questionAndAnswer [ none ] . ( no )

input . ( who played the original scott robinson in neighbours ) => questionAndAnswer [ none ] . ( Darius Perkins )

input . ( how many episodes in the originals season 2 ) => questionAndAnswer [ none ] . ( 22 )



In [34]:

def unicode_to_ascii(s: str) -> str:
  """
  convert unicode to ascii and replaces all accented characters with their equivalent unaccented character (for example, é becomes e)
  """
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')

In [ ]:
def preprocess_sentence(w: str) -> str:
  """
  adds spaces around punctuation and removes all non-alphanumeric characters and adds start and end tokens
  """
  w = unicode_to_ascii(w.lower().strip()).replace(")"," )").replace("(","( ")
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)
  w = re.sub(r"[^a-zA-Z()[]?.!,¿]+", " ", w)
  w = w.strip()
  w = '<sos> ' + w + ' <eos>'
  return w

In [36]:
OTL_FILE_PATH = "/content/drive/MyDrive/ShineBox/data.otl"
#otl - operartion trigger language

def load_otl_file() -> list[str]:
    with open(OTL_FILE_PATH, "r", encoding="utf-8") as f:
        return f.read().splitlines()

def inputs_targets_tuple(otl_file: str) -> tuple[list[str], list[str]]:
    inputs, targets = [], []
    for line in otl_file:
        input = preprocess_sentence(line.split("=>")[0])
        target = preprocess_sentence(line.split("=>")[1])
        inputs.append(input)
        targets.append(target)
    return inputs, targets

In [3]:
def tokenize(lang: list[str]) -> tuple[np.ndarray, tf.keras.preprocessing.text.Tokenizer]:
  """
  the function does the following:
  1. Take the sentence, for example: "May I borrow this book?"
  2. Find the English word index for each word
  3. Create a tensor with the word indices: [18, 3, 141, 22, 47].
  4. Add a padding to the end of the tensor so it's the same length as the other tensors in the set: [18, 3, 141, 22, 47, 0, 0, 0].
  """
  
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,padding='post')
  return tensor, lang_tokenizer

In [38]:
input_lang, target_lang = inputs_targets_tuple(load_otl_file())

In [39]:
def load_data(num_examples: Union[int, None] = None) -> tuple[np.ndarray, np.ndarray, tf.keras.preprocessing.text.Tokenizer, tf.keras.preprocessing.text.Tokenizer]:
  """"
  1. Load the OTL file and store the inputs and targets in input_lang and target_lang
  2. If num_examples is not None, then truncate the input and target lists to num_examples
  3. Create tokenizers for the input and target languages using the tokenize function
  4. Return the input and target tensors,  the input and target tokenizers
  """
  input_lang, target_lang = inputs_targets_tuple(load_otl_file())
  if(num_examples is not None):
    input_lang = input_lang[:num_examples]
    target_lang = target_lang[:num_examples]
  
  input_tensor, inp_lang_tokenizer = tokenize(input_lang)
  target_tensor, targ_lang_tokenizer = tokenize(target_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer
  

In [40]:
num_examples = 15000
input_tensor, target_tensor, inp_lang, targ_lang = load_data(num_examples)

In [41]:
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]
print(max_length_targ, max_length_inp)
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.05)

141 25


In [42]:
# Configuration 
BUFFER_SIZE = len(input_tensor_train) #number of examples for training
BATCH_SIZE = 128 #number of examples per batch
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE #num of batches
steps_per_epoch_val = len(input_tensor_val)//BATCH_SIZE #num of batches in validation set
embedding_dim = 256   #dimentioality of the embedding space
units = 1024  #number of hidden units
vocab_inp_size = len(inp_lang.word_index)+1 #size of the input vocabulary (add 1 for the 0 padding)
vocab_tar_size = len(targ_lang.word_index)+1 #size of the target vocabulary (add 1 for the 0 padding)

In [43]:
#packing the data from the otl files to tf.data.dataset

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
validation_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_val, target_tensor_val)).shuffle(BUFFER_SIZE)
validation_dataset = validation_dataset.batch(BATCH_SIZE, drop_remainder=True)

example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([128, 25]), TensorShape([128, 141]))

<h2>Building The Models (DP And BAH)</h2>

In [44]:

class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_size):
    super(Encoder, self).__init__()
    self.batch_size = batch_size
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim) #embedding layer
    self.gru = tf.keras.layers.GRU(self.enc_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform') #GRU layer

  def call(self, x, hidden):
    x = self.embedding(x) #embedding layer
    output, state = self.gru(x, initial_state = hidden) #GRU layer
    return output, state
    
  def initialize_hidden_state(self):
    return tf.zeros((self.batch_size, self.enc_units)) #initializing the hidden state

In [45]:
class DotProductAttention(tf.keras.layers.Layer):
  def call(self, query, values):
    query_with_time_axis = tf.expand_dims(query, 1)
    score = query_with_time_axis * values
    score = tf.reduce_sum(score, axis=2)
    score = tf.expand_dims(score, 2)
    attention_weights = tf.nn.softmax(score, axis=1)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [46]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    query_with_time_axis = tf.expand_dims(query, 1)
    score = self.V(tf.nn.tanh( self.W1(query_with_time_axis) + self.W2(values)))
    attention_weights = tf.nn.softmax(score, axis=1)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [47]:
class DecoderWithAttention(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_size, attention_layer = None):
    super(DecoderWithAttention, self).__init__()
    self.batch_size = batch_size  
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim) #embedding layer
    self.gru = tf.keras.layers.GRU(self.dec_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform') #GRU layer
    self.fc = tf.keras.layers.Dense(vocab_size) #dense layer

    self.attention = attention_layer #attention layer

  def call(self, x, hidden, enc_output):
    x = self.embedding(x) #embedding layer
    attention_weights = None
    if self.attention: #if attention layer is not None
      context_vector, attention_weights = self.attention(hidden, enc_output) #calculate the context vector and the attention weights
      x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1) #concatenate the context vector with the input

    output, state = self.gru(x, initial_state = hidden) #GRU layer

    output = tf.reshape(output, (-1, output.shape[2])) #reshape the output

    x = self.fc(output) #dense layer
 
    return x, state, attention_weights #return the output, the hidden state and the attention weights

<h2>Building the Training framework</h2>

In [48]:

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')  
optimizer = tf.keras.optimizers.Adam() #optimizer - Adam

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))  #masking the loss
  loss_ = loss_object(real, pred) #calculating the loss
  mask = tf.cast(mask, dtype=loss_.dtype) #casting the mask to the same type as the loss
  loss_ *= mask #applying the mask to the loss
  return tf.reduce_mean(loss_) #returning the mean loss

In [49]:
def get_train_step_func(): #function to return the train step (decorated with tf.function)

  @tf.function
  def train_step(inp, targ, enc_hidden, encoder, decoder):
    loss = 0

    with tf.GradientTape() as tape:  # using the gradient tape to calculate the gradients
      enc_output, enc_hidden = encoder(inp, enc_hidden) #passing the input to the encoder

      dec_hidden = enc_hidden #setting the decoder hidden state to the encoder hidden state

      dec_input = tf.expand_dims([targ_lang.word_index['<sos>']] * BATCH_SIZE, 1) #setting the decoder input to the <sos> token

      for t in range(1, targ.shape[1]): #for each token in the target
        predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output) #passing the decoder input, the decoder hidden state and the encoder output to the decoder

        loss += loss_function(targ[:, t], predictions) #calculating the loss

        dec_input = tf.expand_dims(targ[:, t], 1) #setting the decoder input to the target token

    batch_loss = (loss / int(targ.shape[1])) #calculating the mean loss

    variables = encoder.trainable_variables + decoder.trainable_variables #getting the trainable variables

    optimizer.build(variables) #building the optimizer

    gradients = tape.gradient(loss, variables) #calculating the gradients

    optimizer.apply_gradients(zip(gradients, variables)) #applying the gradients

    return batch_loss #returning the mean loss
    
  return train_step #returning the train step

In [50]:
def caculate_validation_loss(inp, targ, enc_hidden, encoder, decoder):
  loss = 0 
  enc_output, enc_hidden = encoder(inp, enc_hidden)
  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<sos>']] * BATCH_SIZE, 1)

  # Teacher forcing - feeding the target as the next input
  for t in range(1, targ.shape[1]):
    predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
    loss += loss_function(targ[:, t], predictions)
    dec_input = tf.expand_dims(targ[:, t], 1)

  loss = loss / int(targ.shape[1])
  return loss

In [51]:
def training_seq2seq(epochs, attention):
  encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE) #creating the encoder 
  decoder = DecoderWithAttention(vocab_tar_size, embedding_dim, units, BATCH_SIZE, attention) #creating the decoder (with attention that is passed as a parameter)
  train_step_func = get_train_step_func() #getting the train step function
  training_loss: list[float] = []
  validation_loss: list[float] = []

  for epoch in range(epochs): #for each epoch
    start: float = time.time()
    enc_hidden = encoder.initialize_hidden_state() #initializing the encoder hidden state
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)): #for each batch and input and target in the dataset (unpacking the dataset using the take function)
      batch_loss = train_step_func(inp, targ, enc_hidden, encoder, decoder) #calculating the batch loss
      total_loss += batch_loss

      if batch % 10 == 0: #printing the loss every 10 batches
        print('Epoch {} Batch {} Loss {:.8f}'.format(epoch + 1, batch, batch_loss))
        
    #calculating the validation loss
    enc_hidden = encoder.initialize_hidden_state()
    total_val_loss = 0
    for (batch, (inp, targ)) in enumerate(validation_dataset.take(steps_per_epoch)):
      val_loss = caculate_validation_loss(inp, targ, enc_hidden, encoder, decoder)
      total_val_loss += val_loss


    training_loss.append(total_loss / steps_per_epoch) #appending the mean loss to the training loss list
    validation_loss.append(total_val_loss / steps_per_epoch_val) #appending the mean loss to the validation loss list


    print('Epoch {} Loss {:.8f} Validation Loss {:.8f}'.format(epoch + 1, training_loss[-1], validation_loss[-1])) #printing the mean loss and the mean validation loss
    print(f'Time taken for 1 epoch {time.time() - start} sec\n') #printing the time taken for the epoch
  return encoder, decoder, training_loss, validation_loss

<h2>Training and Predicting</h2>

In [52]:
import pickle

def save_tokenizers(): #function to save the tokenizers
    with open('/content/drive/MyDrive/ShineBox/inp_lang.pickle', 'wb') as handle:
        pickle.dump(inp_lang, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('/content/drive/MyDrive/ShineBox/targ_lang.pickle', 'wb') as handle:
        pickle.dump(targ_lang, handle, protocol=pickle.HIGHEST_PROTOCOL)

save_tokenizers()

In [ ]:
epochs = 150
attention = DotProductAttention()
print("Running seq2seq model with dot product attention")
encoder_dp, decoder_dp, training_loss, validation_loss = training_seq2seq(epochs, attention)

encoder_dp.save_weights("/content/drive/MyDrive/ShineBox/DP/encoder_dp")
decoder_dp.save_weights('/content/drive/MyDrive/ShineBox/DP/decoder_dp')


In [ ]:
import matplotlib.pyplot as plt


# Plotting the graph
plt.plot(range(len(training_loss)), training_loss)

# Setting the x-axis label
plt.xlabel('Epoch')

# Setting the y-axis label
plt.ylabel('Training loss')

plt.title('Training Loss Graph ( Dot Product Attention Mechanism )')


# Displaying the plot
plt.show()

In [ ]:
epochs = 150
attention = BahdanauAttention(units)
print("Running seq2seq model with Bahdanau attention")
encoder_bah, decoder_bah, training_loss, validation_loss = training_seq2seq(epochs, attention)


encoder_bah.save_weights("/content/drive/MyDrive/ShineBox/BAH/encoder_bah")
decoder_bah.save_weights('/content/drive/MyDrive/ShineBox/BAH/decoder_bah')



In [ ]:
import matplotlib.pyplot as plt


# Plotting the graph
plt.plot(range(len(training_loss)), training_loss)

# Setting the x-axis label
plt.xlabel('Epoch')

# Setting the y-axis label
plt.ylabel('Training loss')

plt.title('Training Loss Graph ( Dot Product Attention Mechanism )')


# Displaying the plot
plt.show()

In [ ]:

def predict(sentence, encoder, decoder):
  attention_plot = np.zeros((max_length_targ, max_length_inp)) #creating an attention plot

  sentence = preprocess_sentence(sentence) #preprocessing the sentence

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')] #creating the inputs using the input language tokenizer
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')  #padding the inputs
  inputs = tf.convert_to_tensor(inputs) #converting the inputs to a tensor

  result = ''

  hidden = [tf.zeros((1, units))] #creating the hidden state
  enc_out, enc_hidden = encoder(inputs, hidden) #getting the encoder output and the encoder hidden state

  dec_hidden = enc_hidden #setting the decoder hidden state to the encoder hidden state
  dec_input = tf.expand_dims([targ_lang.word_index['<sos>']], 0) #setting the decoder input to the start of sentence token

  for t in range(max_length_targ): #for each word in the target language
    predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out) #getting the predictions, the decoder hidden state and the attention weights

    predicted_id = tf.argmax(predictions[0]).numpy() #getting the predicted id

    result += targ_lang.index_word[predicted_id] + ' ' #adding the predicted word to the result

    if targ_lang.index_word[predicted_id] == '<eos>': #if the predicted word is the end of sentence token then return the result and the sentence
      return result, sentence

    dec_input = tf.expand_dims([predicted_id], 0) #setting the decoder input to the predicted id

  return result, sentence #returning the result and the sentence if the end of sentence token is not predicted after the maximum length of the target language

In [ ]:

result, sentence = predict(f"input . ( {input()} )", encoder_bah, decoder_bah)
print('Input: %s' % (sentence))
print('Predicted translation: {}'.format(result))
     

